---
toc: false
comments: true
layout: post
title: Sprint 5 Review
description: Review of sprint 5 for review
permalink: /csp/sprint5/review
courses: { csp: {week: 18} }
---

## Intro requests for Review
- Purpose of Program: To allow users to share their cars and talk about their dream cars with similar interest individuals. Also to get help from knowledgable individuals.
- Purpose of Feature: To allow users to send their VIN to us to use on various other parts of the website.


## Input/Output requests (demo)
- Will do this live

## List Requests. Use of list, dictionaries, and database
- The .md file uses the API to get a response for each of the methods: post, get, put, delete. The post runs a refresh vehicles function with then uses the get to update the table. The get just updates the table. The put sends a request to the server and when the server responds back it updates the table. The delete removes a row from the database and table.
- The query in the API code looks for a SQLalchemy model
- in the _CRUD class I use the create, read, update, and delete methods to create lists in the database. The methods adds the data to the different dictionaries, like a certain make for the make column, which all add up to create a list for the user.


## Algorithmic Code Request

- This class uses the flask_restful library to handle all of the _CRUD methods

- Parameters: body of request; { "vin": "12345678901234567" } This is an example where the post method accepts a json payload
- Return type: returns json via jsonify. Seen with response messages like {"message": "Vehicle not found"}

- The PUT method uses sequencing, selection and iteration. It uses sequencing because it flows from reading, to fetching to updating. It uses selection because it uses checks for user authentication, making sure inputs are correct (like not longer than 17 characters etc) and database queries. It also uses iteration by taking a response from the NHTSA API and extracting data.
- The put method is below:

## Call to Algorithm Request
Example (of post) is below:

- The code above calls for the api and gets a response in json. This can then be used to display on the DOM
- return/response handling can be seen with successful or failed requests. With a success UI elements will be updates, while on a failure the elements will not be updated except for an error
- Changing data or method triggers different responses, as seen with a successful VIN post and an unsuccessful one. Submitting a unique and real VIN will be met with a success message, while a VIN that isn't real or isn't 17 characters long for instance will be met with an error
- Normal condition: {"message": "Vehicle added successfully", "vehicle": {"vin": "123...17", "make": "Toyota", "model": "Corolla"}}
- Error condition: {"message": "VIN must be 17 characters long"}